In [373]:
# imports
import pandas as pd
import numpy as np

In [374]:
# read the data
class_data, stats_data = pd.read_csv('address_clust.csv'), pd.read_csv('address_stats.csv')

In [375]:
# merge dataframe, name 'zero' claster and drop excess columns
data_all = class_data.merge(stats_data, how='outer').drop(columns=['id','address_id']).fillna({'cluster_id':0})

In [376]:
# group data by transaction and count sent & recived money
b = data_all.groupby(['transaction_id', 'cluster_id']).agg({'received':'sum', 'sent' : 'sum'}).reset_index()

In [377]:
# create dataframe with 1 row : all transaction information
# in case of many clasters better use pivot
zero = b.loc[b.cluster_id == 0.0]
one = b.loc[b.cluster_id == 1.0]
two = b.loc[b.cluster_id == 2.0]

recive_sent = one.merge(two, how='outer', on='transaction_id').merge(zero, how='outer', on='transaction_id').\
fillna({'received_x':0, 'sent_x':0, 'received_y':0, 'sent_y':0, 'received':0, 'sent':0})
recive_sent.head()

,transaction_id,cluster_id_x,received_x,sent_x,cluster_id_y,received_y,sent_y,cluster_id,received,sent
0,19162323,1.0,1000000.0,0.0,NaN,0.0,0.0,0.0,1821052.0,2871052.0
1,19166856,1.0,0.0,1000000.0,NaN,0.0,0.0,0.0,990000.0,0.0
2,19174421,1.0,200000.0,0.0,NaN,0.0,0.0,0.0,4590000.0,4800000.0
3,19190163,1.0,0.0,200000.0,NaN,0.0,0.0,0.0,150000.0,0.0
4,19205422,1.0,1000000.0,0.0,NaN,0.0,0.0,0.0,1671052.0,2721052.0


In [378]:
#__count money gained from one claster to other

def money_clasters(data, new_column_name, claster_from, claster_to, other_claster_to):
    data.loc[:, new_column_name] = 0
    data.loc[(data[claster_from] > 0) & (data[claster_to] > 0),
             [new_column_name]] = data.loc[:, claster_from] - data.loc[:, other_claster_to]
    

In [379]:
money_clasters(recive_sent, 'one_to_two', 'sent_x', 'received_y', 'received')
money_clasters(recive_sent, 'two_to_one', 'sent_y', 'received_x', 'received')
money_clasters(recive_sent, 'zero_to_two', 'sent', 'received_y', 'received_x')
money_clasters(recive_sent, 'two_to_zero', 'sent_y', 'received', 'received_x')

In [380]:
# count fee

# 1-claster fee
recive_sent.loc[:, 'one_fee'] = 0
recive_sent.loc[(recive_sent.sent_x > 0),
['one_fee']] = recive_sent.loc[:, 'sent_x'] - (recive_sent.loc[:, 'received_y'] + recive_sent.loc[:, 'received'] )

# 2-claster fee
recive_sent.loc[:, 'two_fee'] = 0
recive_sent.loc[(recive_sent.sent_y > 0),
['two_fee']] = recive_sent.loc[:, 'sent_y'] - (recive_sent.loc[:, 'received_x'] + recive_sent.loc[:, 'received'] )

In [381]:
# count sum and change satoshi to bitcoin

bitcoin_spent = recive_sent.loc[:, ['one_to_two', 'two_to_one', 'zero_to_two',
                    'two_to_zero','one_fee', 'two_fee']].sum().to_frame().rename(columns={0:'bitcoin'})
bitcoin_spent.loc[:, 'bitcoin'] = bitcoin_spent.loc[:, 'bitcoin'] / 10**8
bitcoin_spent

,bitcoin
one_to_two,149.948516
two_to_one,1026.372663
zero_to_two,26.036068
two_to_zero,4.876490
one_fee,210.467653
two_fee,890.552913
